In [1]:
import robosuite 
import dexmimicgen  

import h5py 
import imageio 
import numpy as np
import time 
import os
import json

'''

1. Loading dual robot environment
'''

def get_env_metadata_from_dataset(dataset_path, ds_format="robomimic"):
    """
    Retrieves env metadata from dataset.

    Args:
        dataset_path (str): path to dataset

    Returns:
        env_meta (dict): environment metadata. Contains 3 keys:

            :`'env_name'`: name of environment
            :`'type'`: type of environment, should be a value in EB.EnvType
            :`'env_kwargs'`: dictionary of keyword arguments to pass to environment constructor
    """
    dataset_path = os.path.expanduser(dataset_path)
    f = h5py.File(dataset_path, "r")
    if ds_format == "robomimic":
        env_meta = json.loads(f["data"].attrs["env_args"])
    else:
        raise ValueError
    f.close()
    return env_meta

dataset_path = "/home/vishal/Volume_E/Active/Gap_year/grand-project-2025/deepak/dexmimicgen/datasets/two_arm_box_cleanup.hdf5"

env_meta = get_env_metadata_from_dataset(dataset_path)
print(env_meta)

write_to_video = False

env_kwargs = env_meta['env_kwargs']
env_kwargs["env_name"] = env_meta["env_name"]
env_kwargs["has_renderer"] = True
env_kwargs["renderer"] = "mjviewer"
env_kwargs["has_offscreen_renderer"] = write_to_video # False # write_video
env_kwargs["use_camera_obs"] = False

env_kwargs.pop("env_lang")

env = robosuite.make(**env_kwargs)

[robosuite WARNING] No private macro file found! (macros.py:57)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:58)
[robosuite WARNING] To setup, run: python /home/vishal/Volume_E/Active/Gap_year/grand-project-2025/deepak/robosuite/robosuite/scripts/setup_macros.py (macros.py:59)
[robosuite WARNING] Could not import robosuite_models. Some robots may not be available. If you want to use these robots, please install robosuite_models from source (https://github.com/ARISE-Initiative/robosuite_models) or through pip install. (__init__.py:30)
[robosuite WARNING] Could not load the mink-based whole-body IK. Make sure you install related import properly, otherwise you will not be able to use the default IK controller setting for GR1 robot. (__init__.py:40)


{'env_name': 'TwoArmBoxCleanup', 'env_version': '1.5.1', 'type': 1, 'env_kwargs': {'robots': ['PandaDexRH', 'PandaDexLH'], 'controller_configs': {'type': 'BASIC', 'body_parts': {'right': {'type': 'OSC_POSE', 'input_max': 1, 'input_min': -1, 'output_max': [0.05, 0.05, 0.05, 0.5, 0.5, 0.5], 'output_min': [-0.05, -0.05, -0.05, -0.5, -0.5, -0.5], 'kp': 150, 'damping_ratio': 1, 'impedance_mode': 'fixed', 'kp_limits': [0, 300], 'damping_ratio_limits': [0, 10], 'position_limits': None, 'orientation_limits': None, 'uncouple_pos_ori': True, 'input_type': 'delta', 'input_ref_frame': 'base', 'interpolation': None, 'ramp_ratio': 0.2, 'gripper': {'type': 'GRIP', 'use_action_scaling': False}}}}, 'translucent_robot': False, 'env_configuration': 'parallel', 'reward_shaping': False, 'camera_names': ['agentview', 'robot0_eye_in_hand', 'robot1_eye_in_hand'], 'camera_heights': 84, 'camera_widths': 84, 'has_renderer': False, 'has_offscreen_renderer': True, 'ignore_done': True, 'use_object_obs': True, 'use_

In [2]:
import pose_utils_vishal_from_mimicgen as PoseUtils
import robosuite.utils.transform_utils as T

In [3]:
'''
Extracting a demonstration from the dataset
'''

data = h5py.File(dataset_path, "r")
print(f"Keys in the dataset: {data['data'].keys()}")
print(f"Number of demonstrations in the dataset: {len(data['data'].keys())}")
demo_id = 'demo_15'
print(f"Length of demo {demo_id}: {data['data'][demo_id]['obs']['robot0_eef_pos'].shape[0]}")
print(f"Observations shape: {data['data'][demo_id]['obs'].keys()}")
print(f"Actions shape: {data['data'][demo_id]['actions'].shape}")
print(f"Actions: {data['data'][demo_id]['actions']}")
print(f"Actions shape: {data['data'][demo_id]['actions'].shape}")
print(f"Action at step 0: {data['data'][demo_id]['actions'][0]}")

action_trajectory = data['data'][demo_id]['actions']
obs_trajectory = data['data'][demo_id]['obs']
states_trajectory = data['data'][demo_id]['states']

eef_poses = data['data'][demo_id]['datagen_info']['eef_pose']
gripper_actions = data['data'][demo_id]['datagen_info']['gripper_action']
target_eef_poses = data['data'][demo_id]['datagen_info']['target_pose']

Keys in the dataset: <KeysViewHDF5 ['demo_0', 'demo_1', 'demo_10', 'demo_100', 'demo_1000', 'demo_1001', 'demo_1002', 'demo_1003', 'demo_1004', 'demo_1005', 'demo_1006', 'demo_1007', 'demo_1008', 'demo_1009', 'demo_101', 'demo_1010', 'demo_1011', 'demo_1012', 'demo_1013', 'demo_1014', 'demo_1015', 'demo_102', 'demo_103', 'demo_104', 'demo_105', 'demo_106', 'demo_107', 'demo_108', 'demo_109', 'demo_11', 'demo_110', 'demo_111', 'demo_112', 'demo_113', 'demo_114', 'demo_115', 'demo_116', 'demo_117', 'demo_118', 'demo_119', 'demo_12', 'demo_120', 'demo_121', 'demo_122', 'demo_123', 'demo_124', 'demo_125', 'demo_126', 'demo_127', 'demo_128', 'demo_129', 'demo_13', 'demo_130', 'demo_131', 'demo_132', 'demo_133', 'demo_134', 'demo_135', 'demo_136', 'demo_137', 'demo_138', 'demo_139', 'demo_14', 'demo_140', 'demo_141', 'demo_142', 'demo_143', 'demo_144', 'demo_145', 'demo_146', 'demo_147', 'demo_148', 'demo_149', 'demo_15', 'demo_150', 'demo_151', 'demo_152', 'demo_153', 'demo_154', 'demo_155'

In [4]:
def just_reset(env, state):
    # The following resets the state
    env.reset()
    # env.sim.set_state_from_flattened(state)
    # env.sim.forward()

    # This part is just to visualize the state after resetting
    env.render()
    obfull = env.step(np.zeros(env.action_dim))
    env.render()
    return obfull

def reset_to_state(env, state):
    # The following resets the state
    env.reset()
    env.sim.set_state_from_flattened(state)
    env.sim.forward()

    # This part is just to visualize the state after resetting
    env.render()
    obfull = env.step(np.zeros(env.action_dim))
    env.render()
    return obfull

In [5]:
def reset_to(env, state):
    """
    Reset to a specific simulator state.

    Args:
        state (dict): current simulator state that contains one or more of:
            - states (np.ndarray): initial state of the mujoco environment
            - model (str): mujoco scene xml

    Returns:
        observation (dict): observation dictionary after setting the simulator state (only
            if "states" is in @state)
    """
    should_ret = False
    if "model" in state:
        if state.get("ep_meta", None) is not None:
            # set relevant episode information
            ep_meta = json.loads(state["ep_meta"])
        else:
            ep_meta = {}
        if hasattr(env, "set_attrs_from_ep_meta"):  # older versions had this function
            env.set_attrs_from_ep_meta(ep_meta)
        elif hasattr(env, "set_ep_meta"):  # newer versions
            env.set_ep_meta(ep_meta)
        # this reset is necessary.
        # while the call to env.reset_from_xml_string does call reset,
        # that is only a "soft" reset that doesn't actually reload the model.
        env.reset()
        robosuite_version_id = int(robosuite.__version__.split(".")[1])
        if robosuite_version_id <= 3:
            from robosuite.utils.mjcf_utils import postprocess_model_xml

            xml = postprocess_model_xml(state["model"])
        else:
            # v1.4 and above use the class-based edit_model_xml function
            xml = env.edit_model_xml(state["model"])

        env.reset_from_xml_string(xml)
        env.sim.reset()
        # hide teleop visualization after restoring from model
        # env.sim.model.site_rgba[env.eef_site_id] = np.array([0., 0., 0., 0.])
        # env.sim.model.site_rgba[env.eef_cylinder_id] = np.array([0., 0., 0., 0.])
    if "states" in state:
        env.sim.set_state_from_flattened(state["states"])
        env.sim.forward()
        should_ret = True

    # update state as needed
    if hasattr(env, "update_sites"):
        # older versions of environment had update_sites function
        env.update_sites()
    if hasattr(env, "update_state"):
        # later versions renamed this to update_state
        env.update_state()

    # if should_ret:
    #     # only return obs if we've done a forward call - otherwise the observations will be garbage
    #     return get_observation()
    return None

In [6]:
ep = demo_id
states = data["data/{}/states".format(ep)][()]
initial_state = dict(states=states[0])
initial_state["model"] = data["data/{}".format(ep)].attrs["model_file"]
# if args.use_current_model:
initial_state["model"] = env.sim.model.get_xml()
initial_state["ep_meta"] = data["data/{}".format(ep)].attrs.get("ep_meta", None)

In [7]:
initial_state

{'states': array([ 0.00000000e+00, -3.25872610e-03,  2.28353831e-01,  1.36325441e-02,
        -2.61769628e+00, -1.75559264e-03,  2.92195030e+00,  7.87831973e-01,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -2.27487457e-02,  2.03329592e-01, -3.71702632e-02, -2.64133752e+00,
         2.84979366e-02,  2.97152396e+00,  8.11196805e-01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -9.38104775e-03,
        -1.50000000e-01,  7.76000000e-01,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -8.00181799e-03,  2.00000000e-01,
         7.25000000e-01,  1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
  

In [8]:
reset_to(env, initial_state)

In [9]:
'''
Replay a demonstration
'''
# reset_to_state(env, states_trajectory[0])
# just_reset(env, states_trajectory[25])

for i in range(action_trajectory.shape[0]):
    action = action_trajectory[i]
    obs, reward, done, info = env.step(action)
    env.render()
    time.sleep(0.05)

In [10]:
target_eef_poses[0], PoseUtils.unmake_pose(target_eef_poses[i])

(array([[[-0.99597645, -0.08945754, -0.00531572, -0.11941785],
         [-0.00507369, -0.00293271,  0.99998283, -0.24456583],
         [-0.08947159,  0.99598632,  0.00246703,  1.09345057],
         [ 0.        ,  0.        ,  0.        ,  1.        ]],
 
        [[ 0.98700808, -0.13165436, -0.09209875, -0.12173107],
         [-0.09198753,  0.00693941, -0.99573598,  0.22776699],
         [ 0.13173209,  0.99127139, -0.00526131,  1.10272726],
         [ 0.        ,  0.        ,  0.        ,  1.        ]]]),
 (array([[-0.11542568, -0.34429114,  1.08323057],
         [-0.07391641,  0.14067788,  1.08101442]]),
  array([[[-0.93137332, -0.34907672,  0.10338851],
          [ 0.07635363,  0.09037731,  0.99297637],
          [-0.35596891,  0.93272578, -0.05752174]],
  
         [[ 0.98510009, -0.05834673, -0.16178218],
          [-0.1510065 ,  0.1567645 , -0.97602353],
          [ 0.08230949,  0.98591103,  0.14561799]]])))

In [11]:
'''
Generating novel demonstration
'''
obfull = reset_to_state(env, states_trajectory[0])
# obfull = just_reset(env, states_trajectory[25])

max_dpos = 0.05
max_drot = 0.5



In [14]:
reset_to_state(env, states_trajectory[0])

(OrderedDict([('robot0_joint_pos',
               array([-2.34213382e-03,  2.24878498e-01,  1.62645324e-02, -2.61574213e+00,
                      -1.83771826e-03,  2.92742221e+00,  7.85534734e-01])),
              ('robot0_joint_pos_cos',
               array([ 0.99999726,  0.97482121,  0.99986774, -0.86489733,  0.99999831,
                      -0.97715304,  0.7070102 ])),
              ('robot0_joint_pos_sin',
               array([-0.00234213,  0.22298792,  0.01626382, -0.50194881, -0.00183772,
                       0.2125369 ,  0.70720334])),
              ('robot0_joint_vel',
               array([ 0.03047519, -0.10368043,  0.07342554,  0.04399332, -0.00633882,
                       0.18813503, -0.06626978])),
              ('robot0_joint_acc',
               array([ 0.32333245, -0.4269361 ,  0.01775079, -0.51305424, -0.25076689,
                       1.54422078, -0.13497196])),
              ('robot0_eef_pos',
               array([-0.11973459, -0.24348091,  1.09375577])),
  

In [12]:
obfull[0].keys()
obfull[0]['robot0_eef_pos'], obfull[0]['robot0_eef_quat'], obfull[0]['robot1_eef_pos'], obfull[0]['robot1_eef_quat']

(array([-0.11952549, -0.24361156,  1.09561235]),
 array([ 0.99897979,  0.00641267,  0.04465708, -0.00199915]),
 array([-0.12351637,  0.22161456,  1.10078049]),
 array([ 0.9965575 , -0.05281205,  0.06388115,  0.00179824]))

In [13]:
eef_poses[0]

array([[[-0.99707392, -0.07581728, -0.00976386, -0.12154334],
        [-0.00952934, -0.00345441,  0.99994863, -0.24502211],
        [-0.07584711,  0.99711574,  0.00272182,  1.09128778],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],

       [[ 0.98572615, -0.12634151, -0.11127346, -0.12433157],
        [-0.11088725,  0.01010797, -0.99378159,  0.22114966],
        [ 0.12668062,  0.99193531, -0.00404597,  1.10033209],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]])

In [12]:
def eef_to_action(eef_pos, eef_rot_mats, original_action, prev_eef_pos, prev_eef_quat, max_dpos=0.05, max_drot=0.5, cur_rot_from_env=False):
    """
    Convert end-effector pose target actions to simulation-executable action space.

    args:
        eef_pos (np.ndarray): target end-effector position (2, 3)
        eef_rot_mats (np.ndarray): target end-effector orientation as rotation matrices (2, 3, 3)
        original_action (np.ndarray): original action to copy other dimensions from (15,)
        prev_eef_pos (np.ndarray): previous end-effector position (2, 3)
        prev_eef_quat (np.ndarray): previous end-effector orientation as quaternion (2, 4)
        max_dpos (float): maximum position delta
        max_drot (float): maximum rotation delta in axis-angle representation
        cur_rot_from_env (bool): rotation matrices from environment are slightly different from that ones from the dataset.
            If true, these rotation matrices must be adjusted before computing the delta rotation. Otherwise, the delta rotation
            will be incorrect.
    """
    max_dpos = 0.05
    max_drot = 0.5

    action = np.copy(original_action)

    # Set robot action in position space
    action[0:3] = eef_pos[0] - prev_eef_pos[0]
    action[0:3] = np.clip(action[0:3] / max_dpos, -1., 1.)
    action[12:15] = eef_pos[1] - prev_eef_pos[1]
    action[12:15] = np.clip(action[12:15] / max_dpos, -1., 1.)

    # Set robot action in rotation space
    curr_rot = T.quat2mat(prev_eef_quat[0])
    target_rot = eef_rot_mats[0] # T.quat2mat(prev_eef_quat[0])
    if cur_rot_from_env:
        curr_rot[:, [1, 2]] = curr_rot[:, [2, 1]]
        curr_rot = -1 * curr_rot
    delta_rot_mat = target_rot.dot(curr_rot.T)
    delta_quat = T.mat2quat(delta_rot_mat)
    delta_rotation = T.quat2axisangle(delta_quat)
    delta_rotation = np.clip(delta_rotation / max_drot, -1., 1.)
    action[3:6] = delta_rotation  

    curr_rot = T.quat2mat(prev_eef_quat[1])
    target_rot = eef_rot_mats[1] # T.quat2mat(prev_eef_quat[1])
    if cur_rot_from_env:
        curr_rot[:, [1, 2]] = curr_rot[:, [2, 1]]
        curr_rot[:, 1] = -1 * curr_rot[:, 1]
    delta_rot_mat = target_rot.dot(curr_rot.T)
    delta_quat = T.mat2quat(delta_rot_mat)
    delta_rotation = T.quat2axisangle(delta_quat)
    delta_rotation = np.clip(delta_rotation / max_drot, -1., 1.)
    action[15:18] = delta_rotation
    return action



    # dpos = eef_pos - prev_eef_pos
    # dquat = T.quat_mul(eef_quat, T.quat_conjugate(prev_eef_quat))
    # drot = T.quat2axisangle(dquat)

    # # clip the position and rotation deltas
    # if np.linalg.norm(dpos) > max_dpos:
    #     dpos = (dpos / np.linalg.norm(dpos)) * max_dpos
    # if np.linalg.norm(drot) > max_drot:
    #     drot = (drot / np.linalg.norm(drot)) * max_drot
    # # construct the action
    # action = np.concatenate([dpos, drot, [gripper_state]])
    # return action

In [20]:
np.stack([PoseUtils.unmake_pose(eef_poses[i])[0][0], PoseUtils.unmake_pose(eef_poses[i])[0][1]]).shape

(2, 3)

In [13]:
# obs = obfull[0]
reset_to(env, initial_state)

obs = obfull[0].copy()
zero_state = PoseUtils.unmake_pose(eef_poses[0])
obs['robot0_eef_pos'] = zero_state[0][0]
obs['robot0_eef_quat'] = T.mat2quat(zero_state[1][0])
obs['robot1_eef_pos'] = zero_state[0][1]
obs['robot1_eef_quat'] = T.mat2quat(zero_state[1][1])

# for i in range(0, 10): # range(action_trajectory.shape[0]):
for i in range(action_trajectory.shape[0]):
    # action = action_trajectory[i].copy()

    # print(f"\n Step: {i} -------------------- \n")
    # print(f"Obs_eef_pos: {obs['robot0_eef_pos']}, {obs['robot1_eef_pos']}")
    # print(f"dataset eef pos: {PoseUtils.unmake_pose(eef_poses[i])}")
    # print()
    # print(f"Obs_eef_quat_mat: {T.quat2mat(obs['robot0_eef_quat'])}, {T.quat2mat(obs['robot1_eef_quat'])}")
    # print(f"dataset eef quat: {PoseUtils.unmake_pose(eef_poses[i])}")
    action = None
    if i==0:
        action = eef_to_action(
            eef_pos=np.stack([PoseUtils.unmake_pose(target_eef_poses[i])[0][0], PoseUtils.unmake_pose(target_eef_poses[i])[0][1]]),
            eef_rot_mats=np.stack([PoseUtils.unmake_pose(target_eef_poses[i])[1][0], PoseUtils.unmake_pose(target_eef_poses[i])[1][1]]),
            original_action=action_trajectory[i],
            prev_eef_pos=np.stack([obs['robot0_eef_pos'], obs['robot1_eef_pos']]),
            prev_eef_quat=np.stack([obs['robot0_eef_quat'], obs['robot1_eef_quat']]),
            max_dpos=max_dpos,
            max_drot=max_drot,
            cur_rot_from_env=False, #cTrue
        )
    else:
        action = eef_to_action(
            eef_pos=np.stack([PoseUtils.unmake_pose(target_eef_poses[i])[0][0], PoseUtils.unmake_pose(target_eef_poses[i])[0][1]]),
            eef_rot_mats=np.stack([PoseUtils.unmake_pose(target_eef_poses[i])[1][0], PoseUtils.unmake_pose(target_eef_poses[i])[1][1]]),
            original_action=action_trajectory[i],
            prev_eef_pos=np.stack([obs['robot0_eef_pos'], obs['robot1_eef_pos']]),
            prev_eef_quat=np.stack([obs['robot0_eef_quat'], obs['robot1_eef_quat']]),
            max_dpos=max_dpos,
            max_drot=max_drot,
            cur_rot_from_env=True
        )

    # p0 = PoseUtils.unmake_pose(target_eef_poses[i])
    # action[0:3] = p0[0][0] - obs['robot0_eef_pos']
    # action[0:3] = np.clip(action[0:3] / max_dpos, -1., 1.)
    # curr_rot = T.quat2mat(obs['robot0_eef_quat'])
    # # Exchange columns 2 and 3 (index 1 and 2)
    # if i != 0:
    #     # print(f"Swapping curr_rot: {curr_rot}")
    #     curr_rot[:, [1, 2]] = curr_rot[:, [2, 1]]
    #     curr_rot = -1 * curr_rot
    # target_rot = p0[1][0]
    # delta_rot_mat = target_rot.dot(curr_rot.T)
    # delta_quat = T.mat2quat(delta_rot_mat)
    # delta_rotation = T.quat2axisangle(delta_quat)
    # delta_rotation = np.clip(delta_rotation / max_drot, -1., 1.)
    # action[3:6] = delta_rotation  

    # action[12:15] = p0[0][1] - obs['robot1_eef_pos']
    # action[12:15] = np.clip(action[12:15] / max_dpos, -1., 1.)
    # curr_rot = T.quat2mat(obs['robot1_eef_quat'])
    # if i!=0:
    #     curr_rot[:, [1, 2]] = curr_rot[:, [2, 1]]
    #     curr_rot[:, 1] = -1 * curr_rot[:, 1]
    #     # curr_rot = -1 * curr_rot
    # target_rot = p0[1][1]
    # delta_rot_mat = target_rot.dot(curr_rot.T)
    # delta_quat = T.mat2quat(delta_rot_mat)
    # delta_rotation = T.quat2axisangle(delta_quat)
    # delta_rotation = np.clip(delta_rotation / max_drot, -1., 1.)
    # action[15:18] = delta_rotation

    obs, reward, done, info = env.step(action)
    # obfull[0] = obs
    env.render()
    time.sleep(0.05)

    print(f"action[0:6]: {action_trajectory[i][0:6]}, constructed_action[0:6]: {action[0:6]}")
    print(f"action[12:18]: {action_trajectory[i][12:18]}, constructed_action[12:18]: {action[12:18]}")
    print(f"Difference 0:6 and 12:18: {action_trajectory[i][0:6] - action[0:6]}, {action_trajectory[i][12:18] - action[12:18]}")
    print(f"Max Difference 0:6: {np.max(np.abs(action_trajectory[i][0:6] - action[0:6]))}")
    print(f"Max Difference 12:18: {np.max(np.abs(action_trajectory[i][12:18] - action[12:18]))}")
    print(" -------------------------------- ")


action[0:6]: [ 0.04250975  0.00912559  0.04325579 -0.00030339 -0.02734572 -0.00896739], constructed_action[0:6]: [ 0.04250975  0.00912559  0.04325579 -0.0003038  -0.02734572 -0.00896759]
action[12:18]: [ 0.05200997  0.13234648  0.04790337  0.00138189 -0.01038436  0.03850162], constructed_action[12:18]: [ 0.05200997  0.13234648  0.04790337  0.00138215 -0.01038436  0.03850139]
Difference 0:6 and 12:18: [0.00000000e+00 0.00000000e+00 0.00000000e+00 4.07395419e-07
 0.00000000e+00 1.95577741e-07], [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.54600309e-07
  0.00000000e+00  2.34693289e-07]
Max Difference 0:6: 4.073954187333584e-07
Max Difference 12:18: 2.5460030883550644e-07
 -------------------------------- 
action[0:6]: [ 0.0516014   0.03639874  0.03146396  0.00024291 -0.03465641 -0.01275348], constructed_action[0:6]: [ 0.05160139  0.03639897  0.03146395  0.00024295 -0.03465617 -0.01275334]
action[12:18]: [ 0.07534181  0.15855297  0.0740977   0.0021364  -0.01420885  0.03652207], cons

In [25]:
reset_to(env, initial_state)

# obs = obfull[0]
obs = obfull[0].copy()
zero_state = PoseUtils.unmake_pose(eef_poses[0])
obs['robot0_eef_pos'] = zero_state[0][0]
obs['robot0_eef_quat'] = T.mat2quat(zero_state[1][0])
obs['robot1_eef_pos'] = zero_state[0][1]
obs['robot1_eef_quat'] = T.mat2quat(zero_state[1][1])

# for i in range(0, 10): # range(action_trajectory.shape[0]):
for i in range(action_trajectory.shape[0]):
    action = action_trajectory[i].copy()

    print(f"\n Step: {i} -------------------- \n")
    print(f"Obs_eef_pos: {obs['robot0_eef_pos']}, {obs['robot1_eef_pos']}")
    print(f"dataset eef pos: {PoseUtils.unmake_pose(eef_poses[i])}")
    print()
    print(f"Obs_eef_quat_mat: {T.quat2mat(obs['robot0_eef_quat'])}, {T.quat2mat(obs['robot1_eef_quat'])}")
    print(f"dataset eef quat: {PoseUtils.unmake_pose(eef_poses[i])}")


    p0 = PoseUtils.unmake_pose(target_eef_poses[i])
    action[0:3] = p0[0][0] - obs['robot0_eef_pos']
    action[0:3] = np.clip(action[0:3] / max_dpos, -1., 1.)
    curr_rot = T.quat2mat(obs['robot0_eef_quat'])
    # Exchange columns 2 and 3 (index 1 and 2)
    if i != 0:
        # print(f"Swapping curr_rot: {curr_rot}")
        curr_rot[:, [1, 2]] = curr_rot[:, [2, 1]]
        curr_rot = -1 * curr_rot
    target_rot = p0[1][0]
    delta_rot_mat = target_rot.dot(curr_rot.T)
    delta_quat = T.mat2quat(delta_rot_mat)
    delta_rotation = T.quat2axisangle(delta_quat)
    delta_rotation = np.clip(delta_rotation / max_drot, -1., 1.)
    action[3:6] = delta_rotation  

    action[12:15] = p0[0][1] - obs['robot1_eef_pos']
    action[12:15] = np.clip(action[12:15] / max_dpos, -1., 1.)
    curr_rot = T.quat2mat(obs['robot1_eef_quat'])
    if i!=0:
        curr_rot[:, [1, 2]] = curr_rot[:, [2, 1]]
        curr_rot[:, 1] = -1 * curr_rot[:, 1]
        # curr_rot = -1 * curr_rot
    target_rot = p0[1][1]
    delta_rot_mat = target_rot.dot(curr_rot.T)
    delta_quat = T.mat2quat(delta_rot_mat)
    delta_rotation = T.quat2axisangle(delta_quat)
    delta_rotation = np.clip(delta_rotation / max_drot, -1., 1.)
    action[15:18] = delta_rotation

    obs, reward, done, info = env.step(action)
    # obfull[0] = obs
    env.render()
    time.sleep(0.05)

    print(f"action[0:6]: {action_trajectory[i][0:6]}, constructed_action[0:6]: {action[0:6]}")
    print(f"action[12:18]: {action_trajectory[i][12:18]}, constructed_action[12:18]: {action[12:18]}")
    print(f"Difference 0:6 and 12:18: {action_trajectory[i][0:6] - action[0:6]}, {action_trajectory[i][12:18] - action[12:18]}")
    print(f"Max Difference 0:6: {np.max(np.abs(action_trajectory[i][0:6] - action[0:6]))}")
    print(f"Max Difference 12:18: {np.max(np.abs(action_trajectory[i][12:18] - action[12:18]))}")
    print(" -------------------------------- ")



 Step: 0 -------------------- 

Obs_eef_pos: [-0.12154334 -0.24502211  1.09128778], [-0.12433157  0.22114966  1.10033209]
dataset eef pos: (array([[-0.12154334, -0.24502211,  1.09128778],
       [-0.12433157,  0.22114966,  1.10033209]]), array([[[-0.99707392, -0.07581728, -0.00976386],
        [-0.00952934, -0.00345441,  0.99994863],
        [-0.07584711,  0.99711574,  0.00272182]],

       [[ 0.98572615, -0.12634151, -0.11127346],
        [-0.11088725,  0.01010797, -0.99378159],
        [ 0.12668062,  0.99193531, -0.00404597]]]))

Obs_eef_quat_mat: [[-0.99707395 -0.0758172  -0.00976399]
 [-0.00952945 -0.00345464  0.99994862]
 [-0.07584703  0.99711573  0.00272201]], [[ 0.98572613 -0.1263416  -0.11127357]
 [-0.11088734  0.01010812 -0.99378163]
 [ 0.12668072  0.99193531 -0.00404584]]
dataset eef quat: (array([[-0.12154334, -0.24502211,  1.09128778],
       [-0.12433157,  0.22114966,  1.10033209]]), array([[[-0.99707392, -0.07581728, -0.00976386],
        [-0.00952934, -0.00345441,  0.99

In [ ]:
'''
Now, with reset, without trajectory transformation
'''

obfull = just_reset(env, states_trajectory[0])

# obs = obfull[0]
obs = obfull[0].copy()
# zero_state = PoseUtils.unmake_pose(eef_poses[0])
# obs['robot0_eef_pos'] = zero_state[0][0]
# obs['robot0_eef_quat'] = T.mat2quat(zero_state[1][0])
# obs['robot1_eef_pos'] = zero_state[0][1]
# obs['robot1_eef_quat'] = T.mat2quat(zero_state[1][1])

# for i in range(0, 10): # range(action_trajectory.shape[0]):
for i in range(action_trajectory.shape[0]):
    action = action_trajectory[i].copy()

    print(f"\n Step: {i} -------------------- \n")
    print(f"Obs_eef_pos: {obs['robot0_eef_pos']}, {obs['robot1_eef_pos']}")
    print(f"dataset eef pos: {PoseUtils.unmake_pose(eef_poses[i])}")
    print()
    print(f"Obs_eef_quat_mat: {T.quat2mat(obs['robot0_eef_quat'])}, {T.quat2mat(obs['robot1_eef_quat'])}")
    print(f"dataset eef quat: {PoseUtils.unmake_pose(eef_poses[i])}")


    p0 = PoseUtils.unmake_pose(target_eef_poses[i])
    action[0:3] = p0[0][0] - obs['robot0_eef_pos']
    action[0:3] = np.clip(action[0:3] / max_dpos, -1., 1.)
    curr_rot = T.quat2mat(obs['robot0_eef_quat'])
    # Exchange columns 2 and 3 (index 1 and 2)
    # if i != 0:
        # print(f"Swapping curr_rot: {curr_rot}")
    curr_rot[:, [1, 2]] = curr_rot[:, [2, 1]]
    curr_rot = -1 * curr_rot
    target_rot = p0[1][0]
    delta_rot_mat = target_rot.dot(curr_rot.T)
    delta_quat = T.mat2quat(delta_rot_mat)
    delta_rotation = T.quat2axisangle(delta_quat)
    delta_rotation = np.clip(delta_rotation / max_drot, -1., 1.)
    action[3:6] = delta_rotation  

    action[12:15] = p0[0][1] - obs['robot1_eef_pos']
    action[12:15] = np.clip(action[12:15] / max_dpos, -1., 1.)
    curr_rot = T.quat2mat(obs['robot1_eef_quat'])
    # if i!=0:
    curr_rot[:, [1, 2]] = curr_rot[:, [2, 1]]
    curr_rot[:, 1] = -1 * curr_rot[:, 1]
        # curr_rot = -1 * curr_rot
    target_rot = p0[1][1]
    delta_rot_mat = target_rot.dot(curr_rot.T)
    delta_quat = T.mat2quat(delta_rot_mat)
    delta_rotation = T.quat2axisangle(delta_quat)
    delta_rotation = np.clip(delta_rotation / max_drot, -1., 1.)
    action[15:18] = delta_rotation

    obs, reward, done, info = env.step(action)
    # obfull[0] = obs
    env.render()
    time.sleep(0.05)

    print(f"action[0:6]: {action_trajectory[i][0:6]}, constructed_action[0:6]: {action[0:6]}")
    print(f"action[12:18]: {action_trajectory[i][12:18]}, constructed_action[12:18]: {action[12:18]}")
    print(f"Difference 0:6 and 12:18: {action_trajectory[i][0:6] - action[0:6]}, {action_trajectory[i][12:18] - action[12:18]}")
    print(f"Max Difference 0:6: {np.max(np.abs(action_trajectory[i][0:6] - action[0:6]))}")
    print(f"Max Difference 12:18: {np.max(np.abs(action_trajectory[i][12:18] - action[12:18]))}")
    print(" -------------------------------- ")



 Step: 0 -------------------- 

Obs_eef_pos: [-0.11736876 -0.24833262  1.098864  ], [-0.11188253  0.27311571  1.10289345]
dataset eef pos: (array([[-0.12154334, -0.24502211,  1.09128778],
       [-0.12433157,  0.22114966,  1.10033209]]), array([[[-0.99707392, -0.07581728, -0.00976386],
        [-0.00952934, -0.00345441,  0.99994863],
        [-0.07584711,  0.99711574,  0.00272182]],

       [[ 0.98572615, -0.12634151, -0.11127346],
        [-0.11088725,  0.01010797, -0.99378159],
        [ 0.12668062,  0.99193531, -0.00404597]]]))

Obs_eef_quat_mat: [[ 0.99535618  0.00247853  0.0962285 ]
 [ 0.00230789 -0.99999524  0.00188458]
 [ 0.09623275 -0.00165374 -0.99535717]], [[ 0.99266843  0.05070872  0.10971789]
 [ 0.04935149 -0.99866811  0.01505231]
 [ 0.11033504 -0.00952721 -0.99384885]]
dataset eef quat: (array([[-0.12154334, -0.24502211,  1.09128778],
       [-0.12433157,  0.22114966,  1.10033209]]), array([[[-0.99707392, -0.07581728, -0.00976386],
        [-0.00952934, -0.00345441,  0.99

In [52]:
data['data'][demo_id]['datagen_info'].keys()


<KeysViewHDF5 ['eef_pose', 'gripper_action', 'object_poses', 'subtask_term_signals', 'target_pose']>

In [ ]:
data['data'][demo_id]['datagen_info']['object_poses'].keys()

<KeysViewHDF5 ['box', 'lid']>

In [55]:
data['data'][demo_id]['datagen_info']['subtask_term_signals'].keys()

<KeysViewHDF5 ['lid_off_ground']>

In [67]:
data['data'][demo_id]['datagen_info']['subtask_term_signals']['lid_off_ground'][160]

1

In [14]:
'''
Segment trajectory into two segments based on when ['data'][demo_id]['datagen_info']['subtask_term_signals'] changes from 0 to 1
'''
def segment_trajectory(data, demo_id):
    subtask_signals = data['data'][demo_id]['datagen_info']['subtask_term_signals']['lid_off_ground']
    print(subtask_signals)  # Example: [0, 0, 0, 1, 0, 0, 1]
    segments = []
    current_segment = []
    # for i, signal in enumerate(subtask_signals):
    flip_flag = False
    for i in range(len(data['data'][demo_id]['actions'])):
        signal = subtask_signals[i]
        if signal == 1 and current_segment and not flip_flag:
            segments.append(current_segment)
            current_segment = []
            current_segment.append(i)
            flip_flag = True
        elif signal == 1 and current_segment:
            current_segment.append(i)
        elif signal == 0:
            current_segment.append(i)
    if current_segment:
        segments.append(current_segment)
    return segments

segments = segment_trajectory(data, demo_id)
print(segments)  # Output: [[0, 1, 2], [3, 4, 5]]

<HDF5 dataset "lid_off_ground": shape (238,), type "<i8">
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105], [106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209

In [ ]:
'''
Transforming two segments one by one

Process:
1. Transform segment 1 using object pose at start of segment 1
2. Interpolate between starting pose to starting pose of segment 1
3. Transform segment 2 using object pose at start of segment 2
4. Interpolate between end of segment 1 to starting pose of segment 2

'''

from pose_utils_vishal_from_mimicgen import transform_source_data_segment_using_object_pose
from pose_utils_vishal_from_mimicgen import interpolate_poses
from pose_utils_vishal_from_mimicgen import make_pose
from pose_utils_vishal_from_mimicgen import unmake_pose

def transform_two_segments(
        segment1,
        segment2,
        src_obj1_pose,
        cur_obj1_pose,
        src_obj2_pose,
        cur_obj2_pose,
        starting_eef_pose,
    ):
    '''Transform 2 segments of trajectory from source to current based on object poses

    Args:
        segment1 (list): list of eef_poses for segment 1
        segment2 (list): list of eef_poses for segment 2
        src_obj_pose (np.ndarray): source object pose (4, 4)
        cur_obj_pose (np.ndarray): current object pose (4, 4)
    '''
    # Transform segments into their corresponding target object poses
    transformed_segment1 = transform_source_data_segment_using_object_pose(
        src_poses=segment1,
        src_obj_pose=src_obj1_pose,
        cur_obj_pose=cur_obj1_pose,
    )
    transformed_segment2 = transform_source_data_segment_using_object_pose(
        src_poses=segment2,
        src_obj_pose=src_obj2_pose,
        cur_obj_pose=cur_obj2_pose,
    )

    # Interpolate between starting pose to starting pose of segment 1
    interp_poses0, num_steps = interpolated_poses = interpolate_poses(
        start_poses=starting_eef_pose,
        end_poses=transformed_segment1[0],
        num_steps=10,
        step_size=0.1,
    )

    # Interpolate between end of segment 1 to starting pose of segment 2
    interp_poses1, num_steps = interpolated_poses = interpolate_poses(
        start_poses=transformed_segment1[-1],
        end_poses=transformed_segment2[0],
        num_steps=10,
        step_size=0.1,
    )

    overall_trajectory = [interp_poses0[:-1], transformed_segment1, interp_poses1[1:-1], transformed_segment2]
    overall_trajectory = np.concatenate(overall_trajectory, axis=0)
    return overall_trajectory

def get_transformed_segments_for_both_arms(
        segment1,
        segment2,
        src_obj1_pose,
        cur_obj1_pose,
        src_obj2_pose,
        cur_obj2_pose,
        starting_eef_pose,
    ):
    '''

    args:
        segment1 (np.ndarray): (N, 2, 4, 4)
        segment2 (np.ndarray): (M, 2, 4, 4) 
        src_obj1_pose (np.ndarray): (4, 4)
        cur_obj1_pose (np.ndarray): (4, 4)
        src_obj2_pose (np.ndarray): (4, 4)
        cur_obj2_pose (np.ndarray): (4, 4)
        starting_eef_pose (list): list of two eef poses [(4, 4), (4, 4)]
    returns:
        dict: {
            "arm1": {
                "segment1": transformed_segment1 (np.ndarray): (N', 4, 4)
                "segment2": transformed_segment2 (np.ndarray): (M', 4, 4)
                "starting_eef_pose": starting_eef_pose_arm1 (list): [(4, 4)]
            },
            "arm2": {
                "segment1": transformed_segment1 (np.ndarray): (N', 4,  4)
                "segment2": transformed_segment2 (np.ndarray): (M', 4, 4)
                "starting_eef_pose": starting_eef_pose_arm2 (list): [(  4, 4)]
            }
        }
    '''

    arm1_segment1 = segment1[:, 0, :]
    arm1_segment2 = segment2[:, 0, :]
    arm2_segment1 = segment1[:, 1, :]
    arm2_segment2 = segment2[:, 1, :]
    starting_eef_pose_arm1 = [starting_eef_pose[0][0], starting_eef_pose[1][0]]
    starting_eef_pose_arm2 = [starting_eef_pose[0][1], starting_eef_pose[1][1]]

    transformed_arm1_trajectory = transform_two_segments(
        segment1=arm1_segment1,
        segment2=arm1_segment2,
        src_obj1_pose=src_obj1_pose,
        cur_obj1_pose=cur_obj1_pose,
        src_obj2_pose=src_obj2_pose,
        cur_obj2_pose=cur_obj2_pose,
        starting_eef_pose=starting_eef_pose_arm1,
    )

    transformed_arm2_trajectory = transform_two_segments(
        segment1=arm2_segment1,
        segment2=arm2_segment2,
        src_obj1_pose=src_obj1_pose,
        cur_obj1_pose=cur_obj1_pose,
        src_obj2_pose=src_obj2_pose,
        cur_obj2_pose=cur_obj2_pose,
        starting_eef_pose=starting_eef_pose_arm2,
    )

    return {
        "arm1": {
            "segment1": transformed_arm1_trajectory,
            # "segment2": transformed_arm1_trajectory,
            "starting_eef_pose": starting_eef_pose_arm1
        },
        "arm2": {
            "segment1": transformed_arm2_trajectory,
            # "segment2": transformed_arm2_trajectory,
            "starting_eef_pose": starting_eef_pose_arm2
        }
    }


In [ ]:
'''
Executing final transformed trajectory on a new environment end-2-end
'''

# obs = obfull[0]
# reset_to(env, initial_state)
obsfull = just_reset(env, states_trajectory[0])

obs = obfull[0].copy()
# zero_state = PoseUtils.unmake_pose(eef_poses[0])
# obs['robot0_eef_pos'] = zero_state[0][0]
# obs['robot0_eef_quat'] = T.mat2quat(zero_state[1][0])
# obs['robot1_eef_pos'] = zero_state[0][1]
# obs['robot1_eef_quat'] = T.mat2quat(zero_state[1][1])

# for i in range(0, 10): # range(action_trajectory.shape[0]):
for i in range(action_trajectory.shape[0]):
    action = None
    if i==0:
        action = eef_to_action(
            eef_pos=np.stack([PoseUtils.unmake_pose(target_eef_poses[i])[0][0], PoseUtils.unmake_pose(target_eef_poses[i])[0][1]]),
            eef_rot_mats=np.stack([PoseUtils.unmake_pose(target_eef_poses[i])[1][0], PoseUtils.unmake_pose(target_eef_poses[i])[1][1]]),
            original_action=action_trajectory[i],
            prev_eef_pos=np.stack([obs['robot0_eef_pos'], obs['robot1_eef_pos']]),
            prev_eef_quat=np.stack([obs['robot0_eef_quat'], obs['robot1_eef_quat']]),
            max_dpos=max_dpos,
            max_drot=max_drot,
            cur_rot_from_env=False, #cTrue
        )
    else:
        action = eef_to_action(
            eef_pos=np.stack([PoseUtils.unmake_pose(target_eef_poses[i])[0][0], PoseUtils.unmake_pose(target_eef_poses[i])[0][1]]),
            eef_rot_mats=np.stack([PoseUtils.unmake_pose(target_eef_poses[i])[1][0], PoseUtils.unmake_pose(target_eef_poses[i])[1][1]]),
            original_action=action_trajectory[i],
            prev_eef_pos=np.stack([obs['robot0_eef_pos'], obs['robot1_eef_pos']]),
            prev_eef_quat=np.stack([obs['robot0_eef_quat'], obs['robot1_eef_quat']]),
            max_dpos=max_dpos,
            max_drot=max_drot,
            cur_rot_from_env=True
        )

    obs, reward, done, info = env.step(action)
    # obfull[0] = obs
    env.render()
    time.sleep(0.05)

    print(f"action[0:6]: {action_trajectory[i][0:6]}, constructed_action[0:6]: {action[0:6]}")
    print(f"action[12:18]: {action_trajectory[i][12:18]}, constructed_action[12:18]: {action[12:18]}")
    print(f"Difference 0:6 and 12:18: {action_trajectory[i][0:6] - action[0:6]}, {action_trajectory[i][12:18] - action[12:18]}")
    print(f"Max Difference 0:6: {np.max(np.abs(action_trajectory[i][0:6] - action[0:6]))}")
    print(f"Max Difference 12:18: {np.max(np.abs(action_trajectory[i][12:18] - action[12:18]))}")
    print(" -------------------------------- ")


In [34]:
action_trajectory.shape

(238, 24)

In [35]:
action_trajectory[:segments[0][-1]+1, :].shape, eef_poses[:segments[0][-1]+1, :].shape, len(segments[0])

((106, 24), (106, 2, 4, 4), 106)

In [16]:
obsfull = just_reset(env, states_trajectory[0])

In [17]:
'''
New object data
'''
new_lid_pos = env.sim.data.body('lid_obj_root').xpos
new_lid_mat = env.sim.data.body('lid_obj_root').xmat.reshape(3, 3)
new_box_pos = env.sim.data.body('box_obj_root').xpos
new_box_mat = env.sim.data.body('box_obj_root').xmat.reshape(3, 3)
new_lid_pose = make_pose(new_lid_pos, new_lid_mat)
new_box_pose = make_pose(new_box_pos, new_box_mat)

In [18]:
data['data'][demo_id]['datagen_info']['object_poses']['lid'][0]

array([[ 1.        ,  0.        ,  0.        , -0.00800182],
       [ 0.        ,  1.        ,  0.        ,  0.2       ],
       [ 0.        ,  0.        ,  1.        ,  0.725     ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [19]:
src_lid_pos = data['data'][demo_id]['datagen_info']['object_poses']['lid'][0][:3, 3]
src_lid_mat = data['data'][demo_id]['datagen_info']['object_poses']['lid'][0][:3, :3].reshape(3, 3)
src_box_pos = data['data'][demo_id]['datagen_info']['object_poses']['box'][0][:3, 3]
src_box_mat = data['data'][demo_id]['datagen_info']['object_poses']['box'][0][:3, :3].reshape(3, 3)
src_lid_pose = make_pose(src_lid_pos, src_lid_mat)
src_box_pose = make_pose(src_box_pos, src_box_mat)

In [20]:
'''
Prepare source target_eef_poses for two segments
'''
src_segment1 = target_eef_poses[:segments[0][-1]+1]
src_segment2 = target_eef_poses[segments[0][-1]+1:]
starting_eef_pose = unmake_pose(target_eef_poses[0])
print(f"src_segment1: {src_segment1.shape}, src_segment2: {src_segment2.shape}, starting_eef_pose: {starting_eef_pose}")

src_segment1: (106, 2, 4, 4), src_segment2: (132, 2, 4, 4), starting_eef_pose: (array([[-0.11941785, -0.24456583,  1.09345057],
       [-0.12173107,  0.22776699,  1.10272726]]), array([[[-0.99597645, -0.08945754, -0.00531572],
        [-0.00507369, -0.00293271,  0.99998283],
        [-0.08947159,  0.99598632,  0.00246703]],

       [[ 0.98700808, -0.13165436, -0.09209875],
        [-0.09198753,  0.00693941, -0.99573598],
        [ 0.13173209,  0.99127139, -0.00526131]]]))


In [21]:
transformed_traj_dict = get_transformed_segments_for_both_arms(
    segment1=src_segment1,
    segment2=src_segment2,
    src_obj1_pose=src_lid_pose,
    cur_obj1_pose=new_lid_pose,
    src_obj2_pose=src_box_pose,
    cur_obj2_pose=new_box_pose,
    starting_eef_pose=starting_eef_pose,
)

TypeError: transform_source_data_segment_using_object_pose() got an unexpected keyword argument 'src_poses'

In [79]:
transformed_traj_dict.keys(), transformed_traj_dict['arm1'].keys(), transformed_traj_dict['arm2'].keys()

(dict_keys(['arm1', 'arm2']),
 dict_keys(['segment1', 'segment2', 'starting_eef_pose']),
 dict_keys(['segment1', 'segment2', 'starting_eef_pose']))

In [65]:
new_lid_mat

array([1., 0., 0., 0., 1., 0., 0., 0., 1.])

In [ ]:
# a = np.array([1, 2, 3, 4])
# a[:-1]

array([1, 2, 3])

In [42]:
obsfull[0].keys()

odict_keys(['robot0_joint_pos', 'robot0_joint_pos_cos', 'robot0_joint_pos_sin', 'robot0_joint_vel', 'robot0_joint_acc', 'robot0_eef_pos', 'robot0_eef_quat', 'robot0_eef_quat_site', 'robot0_gripper_qpos', 'robot0_gripper_qvel', 'robot1_joint_pos', 'robot1_joint_pos_cos', 'robot1_joint_pos_sin', 'robot1_joint_vel', 'robot1_joint_acc', 'robot1_eef_pos', 'robot1_eef_quat', 'robot1_eef_quat_site', 'robot1_gripper_qpos', 'robot1_gripper_qvel', 'robot0_proprio-state', 'robot1_proprio-state'])

In [61]:

data['data'][demo_id]['datagen_info']['object_poses']['lid'][0]

array([[ 1.        ,  0.        ,  0.        , -0.00800182],
       [ 0.        ,  1.        ,  0.        ,  0.2       ],
       [ 0.        ,  0.        ,  1.        ,  0.725     ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [63]:
env.sim.data.body('lid_obj_root').xpos

array([0.02686183, 0.2       , 0.72476748])

In [58]:
# env.sim.model.body_pos[env.sim.model.body_name2id('object0')]
reset_to(env, initial_state)
data['data'][demo_id]['datagen_info']['object_poses'].keys()

env.sim.data.body('lid_obj_root')

<_MjDataBodyViews
  cacc: array([0.        , 0.        , 0.        , 0.        , 0.        ,
       9.49160762])
  cfrc_ext: array([0., 0., 0., 0., 0., 0.])
  cfrc_int: array([ 4.79629991e-18, -7.65637127e-18,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  2.08721929e-14])
  cinert: array([ 0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0.,  0.])
  crb: array([ 2.42782661e-03,  2.42782661e-03,  4.24375000e-03,  4.47778712e-35,
        1.21122938e-34, -3.87593401e-33,  8.06646416e-19, -2.58126853e-17,
       -6.98226199e-17,  4.65000000e-01])
  cvel: array([0., 0., 0., 0., 0., 0.])
  id: 75
  name: 'lid_obj_root'
  subtree_angmom: array([0., 0., 0.])
  subtree_com: array([-0.00800182,  0.2       ,  0.76032258])
  subtree_linvel: array([0., 0., 0.])
  xfrc_applied: array([0., 0., 0., 0., 0., 0.])
  ximat: array([1., 0., 0., 0., 1., 0., 0., 0., 1.])
  xipos: array([-0.00800182,  0.2       ,  0.725     ])
  xmat: array([1., 0., 0., 0., 1., 0., 0., 0., 1.])
  xpos: array([-0.00800182, 

In [ ]:
'''
Now, with reset, and appropriate trajectory transformation
'''

obfull = just_reset(env, states_trajectory[0])

# obs = obfull[0]
obs = obfull[0].copy()
# zero_state = PoseUtils.unmake_pose(eef_poses[0])
# obs['robot0_eef_pos'] = zero_state[0][0]
# obs['robot0_eef_quat'] = T.mat2quat(zero_state[1][0])
# obs['robot1_eef_pos'] = zero_state[0][1]
# obs['robot1_eef_quat'] = T.mat2quat(zero_state[1][1])

from pose_utils_vishal_from_mimicgen import transform_source_data_segment_using_object_pose

transformed_ee_poses = transform_source_data_segment_using_object_pose(, target_eef_poses, )

# for i in range(0, 10): # range(action_trajectory.shape[0]):
for i in range(action_trajectory.shape[0]):
    action = action_trajectory[i].copy()

    print(f"\n Step: {i} -------------------- \n")
    print(f"Obs_eef_pos: {obs['robot0_eef_pos']}, {obs['robot1_eef_pos']}")
    print(f"dataset eef pos: {PoseUtils.unmake_pose(eef_poses[i])}")
    print()
    print(f"Obs_eef_quat_mat: {T.quat2mat(obs['robot0_eef_quat'])}, {T.quat2mat(obs['robot1_eef_quat'])}")
    print(f"dataset eef quat: {PoseUtils.unmake_pose(eef_poses[i])}")


    p0 = PoseUtils.unmake_pose(target_eef_poses[i])
    action[0:3] = p0[0][0] - obs['robot0_eef_pos']
    action[0:3] = np.clip(action[0:3] / max_dpos, -1., 1.)
    curr_rot = T.quat2mat(obs['robot0_eef_quat'])
    # Exchange columns 2 and 3 (index 1 and 2)
    # if i != 0:
        # print(f"Swapping curr_rot: {curr_rot}")
    curr_rot[:, [1, 2]] = curr_rot[:, [2, 1]]
    curr_rot = -1 * curr_rot
    target_rot = p0[1][0]
    delta_rot_mat = target_rot.dot(curr_rot.T)
    delta_quat = T.mat2quat(delta_rot_mat)
    delta_rotation = T.quat2axisangle(delta_quat)
    delta_rotation = np.clip(delta_rotation / max_drot, -1., 1.)
    action[3:6] = delta_rotation  

    action[12:15] = p0[0][1] - obs['robot1_eef_pos']
    action[12:15] = np.clip(action[12:15] / max_dpos, -1., 1.)
    curr_rot = T.quat2mat(obs['robot1_eef_quat'])
    # if i!=0:
    curr_rot[:, [1, 2]] = curr_rot[:, [2, 1]]
    curr_rot[:, 1] = -1 * curr_rot[:, 1]
        # curr_rot = -1 * curr_rot
    target_rot = p0[1][1]
    delta_rot_mat = target_rot.dot(curr_rot.T)
    delta_quat = T.mat2quat(delta_rot_mat)
    delta_rotation = T.quat2axisangle(delta_quat)
    delta_rotation = np.clip(delta_rotation / max_drot, -1., 1.)
    action[15:18] = delta_rotation

    obs, reward, done, info = env.step(action)
    # obfull[0] = obs
    env.render()
    time.sleep(0.05)

    print(f"action[0:6]: {action_trajectory[i][0:6]}, constructed_action[0:6]: {action[0:6]}")
    print(f"action[12:18]: {action_trajectory[i][12:18]}, constructed_action[12:18]: {action[12:18]}")
    print(f"Difference 0:6 and 12:18: {action_trajectory[i][0:6] - action[0:6]}, {action_trajectory[i][12:18] - action[12:18]}")
    print(f"Max Difference 0:6: {np.max(np.abs(action_trajectory[i][0:6] - action[0:6]))}")
    print(f"Max Difference 12:18: {np.max(np.abs(action_trajectory[i][12:18] - action[12:18]))}")
    print(" -------------------------------- ")


In [37]:
obs['robot0_eef_pos']

array([-0.27634358,  0.43484571,  0.98203987])

In [21]:
p0[1][0]

array([[-0.99597645, -0.08945754, -0.00531572],
       [-0.00507369, -0.00293271,  0.99998283],
       [-0.08947159,  0.99598632,  0.00246703]])

In [12]:
eef_poses.shape

(238, 2, 4, 4)

In [15]:
for i in range(action_trajectory.shape[0]):
    

    action = action_trajectory[i]

    constructed_action = np.zeros_like(action)

    p0 = PoseUtils.unmake_pose(target_eef_poses[i])

    p_cur = PoseUtils.unmake_pose(eef_poses[i])

    constructed_action[0:3] = p0[0][0] - p_cur[0][0]
    constructed_action[0:3] = np.clip(constructed_action[0:3] / max_dpos, -1., 1.)
    curr_rot = p_cur[1][0] # T.quat2mat(obs['robot0_eef_quat'])
    target_rot = p0[1][0]
    delta_rot_mat = target_rot.dot(curr_rot.T)
    delta_quat = T.mat2quat(delta_rot_mat)
    delta_rotation = T.quat2axisangle(delta_quat)
    delta_rotation = np.clip(delta_rotation / max_drot, -1., 1.)
    constructed_action[3:6] = delta_rotation  

    constructed_action[12:15] = p0[0][1] - p_cur[0][1] # obs['robot1_eef_pos']
    constructed_action[12:15] = np.clip(constructed_action[12:15] / max_dpos, -1., 1.)
    curr_rot = p_cur[1][1] # T.quat2mat(obs['robot1_eef_quat'])
    target_rot = p0[1][1]
    delta_rot_mat = target_rot.dot(curr_rot.T)
    delta_quat = T.mat2quat(delta_rot_mat)
    delta_rotation = T.quat2axisangle(delta_quat)
    delta_rotation = np.clip(delta_rotation / max_drot, -1., 1.)
    constructed_action[15:18] = delta_rotation

    print(f"action[0:6]: {action[0:6]}, constructed_action[0:6]: {constructed_action[0:6]}")
    print(f"action[12:18]: {action[12:18]}, constructed_action[12:18]: {constructed_action[12:18]}")
    print(f"Difference 0:6 and 12:18: {action[0:6] - constructed_action[0:6]}, {action[12:18] - constructed_action[12:18]}")
    print(f"Max Difference 0:6: {np.max(np.abs(action[0:6] - constructed_action[0:6]))}")
    print(f"Max Difference 12:18: {np.max(np.abs(action[12:18] - constructed_action[12:18]))}")
    print(" -------------------------------- ")
    # print(f"constructed action: {constructed_action}")
    # print(f"Difference: {action - constructed_action}")

action[0:6]: [ 0.04250975  0.00912559  0.04325579 -0.00030339 -0.02734572 -0.00896739], constructed_action[0:6]: [ 0.04250975  0.00912559  0.04325579 -0.00030339 -0.02734572 -0.00896734]
action[12:18]: [ 0.05200997  0.13234648  0.04790337  0.00138189 -0.01038436  0.03850162], constructed_action[12:18]: [ 0.05200997  0.13234648  0.04790337  0.00138187 -0.01038508  0.03850117]
Difference 0:6 and 12:18: [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.82307155e-09
 -1.86264515e-09 -5.68106771e-08], [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.76951289e-08
 7.15255737e-07 4.58210707e-07]
Max Difference 0:6: 5.681067705154419e-08
Max Difference 12:18: 7.152557373046875e-07
 -------------------------------- 
action[0:6]: [ 0.0516014   0.03639874  0.03146396  0.00024291 -0.03465641 -0.01275348], constructed_action[0:6]: [ 0.0516014   0.03639874  0.03146396  0.00024291 -0.03465665 -0.01275334]
action[12:18]: [ 0.07534181  0.15855297  0.0740977   0.0021364  -0.01420885  0.03652207], const